In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from io import BytesIO
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image
import os
import random
import re
import pandas as pd
import hashlib
from datetime import datetime
import urllib.parse
from json import dumps, loads
from shutil import copy2

In [2]:
os.chdir('/content/drive/MyDrive/research-similarity/Scraping')

In [3]:
def create_directories(base_url, categories, label='images'):
    # create the following dir struct; outputs > base website > categories
    base_dir = os.path.join('output', label, urlparse(base_url).netloc)
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    for category, _ in categories:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

    return base_dir

In [9]:
def get_articles_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    articles_links = []
    remove = [
        'https://www.indiatimes.com/trending/social-relevance/when-will-shradh-begins-in-2024-know-pitru-pakshas-start-and-end-date-heres-all-you-need-641821.html',
        'https://www.indiatimes.com/entertainment/celebs/aparshakti-khurana-reveals-why-he-touched-ayushmann-khurranas-feet-daily-641820.html',
        'https://www.indiatimes.com/trending/social-relevance/bihar-man-folds-hand-in-front-of-snake-idol-in-temple-then-steals-it-641818.html',
        'https://www.indiatimes.com/news/education/google-isnt-the-same-anymore-employee-working-12-hours-says-company-expects-more-reveals-harsh-reality-after-mass-layoffs-641817.html'
    ]

    for a_tag in soup.find_all('a'):
        try:
            href = a_tag['href']
            if href.endswith('html') and not href.endswith('sitemap.html'):
                link = f'https://www.indiatimes.com{href}'
                if link.startswith(url) and link not in articles_links:
                    articles_links.append(link)
        except:
            continue

    return list(set(articles_links) - set(remove))

def article_scrapper(url):
    articles_links = get_articles_links(url)
    data = []

    def helper_scrapper(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')
        try:
            story_main_div = soup.find('div', id='storyMainDiv')
            headline = soup.find('h1').text
            datetime_str = soup.find('span', class_='authorname font-12').text.split(': ')[-1][:-4]
            time = datetime.strptime(datetime_str, "%b %d, %Y, %H:%M")
            featured_img = soup.find('div', class_='article-image').find('img')
            images = [(featured_img.get('alt'), featured_img.get('src'))]

            # for img in soup.find('div', id='article-description-0').find_all('img'):
            #     img_alt = img.get('alt', '')  # Safely get the 'alt' attribute
            #     img_src = img.get('src', '')  # Safely get the 'src' attribute
            #     if  (img_alt, img_src) not in images:
            #         images.append((img_alt, img_src))

            return headline, time, images
        except:
            return None

    for url in articles_links:
        result = helper_scrapper(url)
        if result is not None:
            data.append(result)

    return data

In [5]:
def get_latest_articles(data, n=10):
    seen_headlines = set()
    unique_data = []

    for record in data:
        headline = record[0]
        if headline not in seen_headlines:
            seen_headlines.add(headline)
            unique_data.append(record)

    return sorted(unique_data, key=lambda x: x[1], reverse=True)[:n]

def download_image(img_url, save_dir, img_name):
    try:
        if not img_url.startswith('data:'):
            response = requests.get(img_url)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            width, height = img.size

            # Only save images larger than 100x100 pixels
            if width >= 100 and height >= 100:
                with open(os.path.join(save_dir, img_name), 'wb') as img_file:
                    img_file.write(img_data)
    except:
        pass

def download_images(category_url, save_dir, data):

    with open(os.path.join(save_dir, 'labels.csv'), 'w') as f:
        f.write('image number,alt,article_heading\n')

    records = []

    # parallising the downloads to make it faster
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        headlines = []
        for x, tuple in enumerate(data):
            headline, _, images_list = tuple
            for i, img in enumerate(images_list):
                alt_txt, img_url = img
                if img_url and not img_url.startswith('data:'):
                    img_url = urljoin(category_url, img_url)
                    combined_str = f"{alt_txt}{headline}".encode()
                    img_name = f'image_{x+1}_{i+1}.jpg'
                    records.append(f'{img_name},{alt_txt.replace(",", "")},{headline.replace(",", "")}\n')
                    futures.append(executor.submit(download_image, img_url, save_dir, img_name))

        with open(os.path.join(save_dir, 'labels.csv'), 'a') as f:
            f.writelines(records)

        for future in as_completed(futures):
            future.result()

In [44]:
categories = [
    ('Beauty and Care', 'https://www.indiatimes.com/fashion/beauty-care'),
    ('Gardening', 'https://www.indiatimes.com/lifestyle/gardening'),
    ('Relationships', 'https://www.indiatimes.com/lifestyle/relationships'),
    ('Astrology', 'https://www.indiatimes.com/lifestyle/astrology'),
    ('Sunny Side Up', 'https://www.indiatimes.com/lifestyle/sunny-side-up'),
    ('Art and Culture', 'https://www.indiatimes.com/lifestyle/art-and-culture'),
    ('Bollywood', 'https://www.indiatimes.com/entertainment/bollywood'),
    ('Celebscoop', 'https://www.indiatimes.com/entertainment/celebs'),
    ('Hollywood', 'https://www.indiatimes.com/entertainment/hollywood'),
    ('Originals', 'https://www.indiatimes.com/entertainment/originals'),
    ('Binge', 'https://www.indiatimes.com/entertainment/binge'),
    ('Healthy Living', 'https://www.indiatimes.com/health/healthyliving'),
    ('Recipes', 'https://www.indiatimes.com/health/recipes'),
    ('Inspire', 'https://www.indiatimes.com/health/videos'),
    ('Tips & Tricks', 'https://www.indiatimes.com/health/tips-tricks'),
    ('Buzz', 'https://www.indiatimes.com/health/buzz'),
    ('Investment', 'https://www.indiatimes.com/worth/investment'),
    ('Mutual Funds', 'https://www.indiatimes.com/worth/mutual-funds'),
    ('Apps', 'https://www.indiatimes.com/technology/apps'),
    ('Electric', 'https://www.indiatimes.com/auto/electric'),
    ('Current', 'https://www.indiatimes.com/auto/current'),
    ('Alternate', 'https://www.indiatimes.com/auto/alternative'),
    ('Sports', 'https://www.indiatimes.com/sports'),
    ('World', 'https://www.indiatimes.com/news/world')
]

base_url = 'https://www.indiatimes.com/'

base_dir = create_directories(base_url, categories)

In [45]:
for category, category_url in tqdm(categories, desc='Downloading images for every category'):
    category_dir = os.path.join(base_dir, category)
    try:
        data = article_scrapper(category_url)
        download_images(category_url, category_dir, get_latest_articles(data))
    except:
        print(category)
        continue

In [46]:
def create_file(base_dir):
    for category in os.listdir(base_dir):
        try:
            df = pd.read_csv(f'{base_dir}{category}/labels.csv')
            n, _ = df.shape

            pairs = []

            for i in range(n):
                for j in range(i + 1, n):
                    article_1 = int(re.search(r'\d+(?=_|$)',list(df.iloc[i])[0]).group())
                    article_2 = int(re.search(r'\d+(?=_|$)',list(df.iloc[j])[0]).group())
                    if article_1 != article_2:
                        pairs.append((list(df.iloc[i]), list(df.iloc[j])))

            with open(f'{base_dir.split("/")[2]}_pairs_{category}.csv', 'w') as f:
                for i, pair in enumerate(pairs):
                    p1, p2 = pair
                    img1, alt1, headline1 = p1
                    img2, alt2, headline2 = p2
                    f.write(f'{i+1},{headline1},{headline2}\n')
                    f.write(f',https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/www.indiatimes.com/{urllib.parse.quote(category)}/{img1},https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/www.indiatimes.com/{urllib.parse.quote(category)}/{img2}\n')
                    f.write(f',{alt1},{alt2}\n')

        except:
            continue

In [47]:
create_file('output/images/www.indiatimes.com/')

In [48]:
num_pairs = []

for file in os.listdir():
    if file.startswith('www.indiatimes.com_pairs'):
        try:
            df = pd.read_csv(file)
            num_pairs.append((int(list(df.iloc[-3])[0]), file))
        except:
            continue

In [49]:
sorted_array = sorted(num_pairs, key=lambda x: x[0])
sorted_array

[(45, 'www.indiatimes.com_pairs_Beauty and Care.csv'),
 (45, 'www.indiatimes.com_pairs_Gardening.csv'),
 (45, 'www.indiatimes.com_pairs_Relationships.csv'),
 (45, 'www.indiatimes.com_pairs_Astrology.csv'),
 (45, 'www.indiatimes.com_pairs_Sunny Side Up.csv'),
 (45, 'www.indiatimes.com_pairs_Art and Culture.csv'),
 (45, 'www.indiatimes.com_pairs_Bollywood.csv'),
 (45, 'www.indiatimes.com_pairs_Celebscoop.csv'),
 (45, 'www.indiatimes.com_pairs_Hollywood.csv'),
 (45, 'www.indiatimes.com_pairs_Originals.csv'),
 (45, 'www.indiatimes.com_pairs_Binge.csv'),
 (45, 'www.indiatimes.com_pairs_Healthy Living.csv'),
 (45, 'www.indiatimes.com_pairs_Recipes.csv'),
 (45, 'www.indiatimes.com_pairs_Inspire.csv'),
 (45, 'www.indiatimes.com_pairs_Tips & Tricks.csv'),
 (45, 'www.indiatimes.com_pairs_Buzz.csv'),
 (45, 'www.indiatimes.com_pairs_Investment.csv'),
 (45, 'www.indiatimes.com_pairs_Mutual Funds.csv'),
 (45, 'www.indiatimes.com_pairs_Apps.csv'),
 (45, 'www.indiatimes.com_pairs_Electric.csv'),
 (45,

In [51]:
import random
random.shuffle(sorted_array)
print(sorted_array)

[(45, 'www.indiatimes.com_pairs_World.csv'), (45, 'www.indiatimes.com_pairs_Current.csv'), (45, 'www.indiatimes.com_pairs_Alternate.csv'), (45, 'www.indiatimes.com_pairs_Sunny Side Up.csv'), (45, 'www.indiatimes.com_pairs_Celebscoop.csv'), (45, 'www.indiatimes.com_pairs_Hollywood.csv'), (45, 'www.indiatimes.com_pairs_Recipes.csv'), (45, 'www.indiatimes.com_pairs_Investment.csv'), (45, 'www.indiatimes.com_pairs_Buzz.csv'), (45, 'www.indiatimes.com_pairs_Originals.csv'), (45, 'www.indiatimes.com_pairs_Beauty and Care.csv'), (45, 'www.indiatimes.com_pairs_Apps.csv'), (45, 'www.indiatimes.com_pairs_Art and Culture.csv'), (45, 'www.indiatimes.com_pairs_Gardening.csv'), (45, 'www.indiatimes.com_pairs_Inspire.csv'), (45, 'www.indiatimes.com_pairs_Mutual Funds.csv'), (45, 'www.indiatimes.com_pairs_Binge.csv'), (45, 'www.indiatimes.com_pairs_Bollywood.csv'), (45, 'www.indiatimes.com_pairs_Tips & Tricks.csv'), (45, 'www.indiatimes.com_pairs_Healthy Living.csv'), (45, 'www.indiatimes.com_pairs_Sp